### Import

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from google.colab import drive

### Data Load

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/MyDrive/open/train.csv').drop(columns=['ID'])
test = pd.read_csv('/content/drive/MyDrive/open/test.csv').drop(columns=['ID'])

In [36]:
# 최빈값으로 '알 수 없음' 대체
mode_value = train['시술 당시 나이'].mode()[0]

train.loc[train['시술 당시 나이'] == '알 수 없음', '시술 당시 나이'] = mode_value
test.loc[test['시술 당시 나이'] == '알 수 없음', '시술 당시 나이'] = mode_value

<ipython-input-36-0ad38bf16a5d>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '만18-34세' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test.loc[test['시술 당시 나이'] == '알 수 없음', '시술 당시 나이'] = mode_value


In [38]:
analysis_results = {}
for col in categorical_columns:
    if col in train.columns:
        value_counts = train[col].value_counts(dropna=False)  # NaN 포함하여 개수 계산
        missing_values = train[col].isna().sum()  # 결측값 개수

        analysis_results[col] = {
            "Value Counts": value_counts.to_dict(),
            "Missing Values": missing_values
        }

In [39]:
for col, result in analysis_results.items():
    print(f"컬럼: {col}")
    print("값 개수:", result["Value Counts"])
    print("결측값 개수:", result["Missing Values"])
    print("-" * 50)

컬럼: 시술 시기 코드
값 개수: {'TRDQAZ': 38969, 'TRCMWS': 38090, 'TRYBLT': 36713, 'TRVNRY': 36173, 'TRJXFG': 36031, 'TRZKPL': 35544, 'TRXQMD': 34831}
결측값 개수: 0
--------------------------------------------------
컬럼: 시술 당시 나이
값 개수: {'만18-34세': 102805, '만35-37세': 57780, '만38-39세': 39247, '만40-42세': 37348, '만43-44세': 12253, '만45-50세': 6918}
결측값 개수: 0
--------------------------------------------------
컬럼: 특정 시술 유형
값 개수: {'ICSI': 122368, 'IVF': 91755, 'Unknown': 26939, 'IUI': 6100, 'ICSI:ICSI': 2314, 'ICSI / BLASTOCYST ': 1609, 'IVF / BLASTOCYST': 1248, 'IVF:IVF': 1146, 'ICSI:IVF': 873, 'ICSI / AH': 769, 'IVF:ICSI': 392, 'IVF / AH': 319, 'ICSI:Unknown': 207, 'IVF:Unknown': 100, 'ICI': 96, 'Generic DI': 71, 'IVI': 23, 'ICSI / BLASTOCYST:IVF / BLASTOCYST': 6, 'ICSI / BLASTOCYST :IVF / BLASTOCYST': 4, 'FER': 3, 'IVF / AH:ICSI / AH': 2, nan: 2, 'GIFT': 2, 'ICSI / AH:Unknown': 2, 'ICSI / BLASTOCYST :ICSI': 1}
결측값 개수: 2
--------------------------------------------------
컬럼: 배란 자극 여부
값 개수: {1: 197720, 0: 5863

### Data Pre-processing

In [40]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [41]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "특정 시술 유형",
    "배란 자극 여부",
    "단일 배아 이식 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "난자 채취 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
]

In [42]:
# categorical_columns에 포함된 컬럼만 유지
X_train = X_train[categorical_columns]
X_val = X_val[categorical_columns]
test = test[categorical_columns]

# 확인: 남아있는 컬럼 출력
print("학습 데이터 남아있는 컬럼:", X_train.columns.tolist())
print("검증 데이터 남아있는 컬럼:", X_val.columns.tolist())
print("테스트 데이터 남아있는 컬럼:", test.columns.tolist())

학습 데이터 남아있는 컬럼: ['시술 시기 코드', '시술 당시 나이', '특정 시술 유형', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 기증자 나이', '정자 기증자 나이', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', '난자 채취 경과일', '난자 혼합 경과일', '배아 이식 경과일']
검증 데이터 남아있는 컬럼: ['시술 시기 코드', '시술 당시 나이', '특정 시술 유형', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', 

In [11]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.3 MB/s eta 0:00:00


In [43]:
from category_encoders import TargetEncoder

encoder = TargetEncoder(cols=categorical_columns)
X_train_encoded = encoder.fit_transform(X_train, y_train)
X_val_encoded = encoder.transform(X_val)
test_encoded = encoder.transform(test)

In [44]:
models = [
    ('random_forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('adaboost', AdaBoostClassifier(n_estimators=100, random_state=42)),
    ('gradient_boosting', GradientBoostingClassifier(n_estimators=100, random_state=42)),
    ('logistic_regression', LogisticRegression(max_iter=500))
]

In [45]:
from category_encoders import TargetEncoder

encoder = TargetEncoder(cols=categorical_columns)
X_train = encoder.fit_transform(X_train, y_train)
X_val = encoder.transform(X_val)
test = encoder.transform(test)

print(X_train.dtypes)  # 모든 컬럼이 숫자인지 확인

In [46]:

# from category_encoders import TargetEncoder

# # Target Encoding 적용
# encoder = TargetEncoder(cols=['시술 유형'])
# X_train = encoder.fit_transform(X_train, y_train)
# X_val = encoder.transform(X_val)
# test = encoder.transform(test)

# # 변환 후 데이터 타입 확인
# print(X_train.dtypes)  # 모든 컬럼이 float64인지 확인


ValueError: X does not contain the columns listed in cols

In [50]:
from category_encoders import TargetEncoder

# Target Encoding 적용
encoder = TargetEncoder(cols=['시술 시기 코드', '시술 당시 나이', '특정 시술 유형', '배란 자극 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '난자 기증자 나이', '정자 기증자 나이'])
X_train = encoder.fit_transform(X_train, y_train)
X_val = encoder.transform(X_val)
test = encoder.transform(test)

# 변환 후 데이터 타입 확인
print(X_train.dtypes)  # 모든 컬럼이 float64인지 확인

시술 시기 코드            float64
시술 당시 나이            float64
특정 시술 유형            float64
배란 자극 여부            float64
단일 배아 이식 여부         float64
착상 전 유전 진단 사용 여부    float64
남성 주 불임 원인          float64
남성 부 불임 원인          float64
여성 주 불임 원인          float64
여성 부 불임 원인          float64
부부 주 불임 원인          float64
부부 부 불임 원인          float64
불명확 불임 원인           float64
불임 원인 - 난관 질환       float64
불임 원인 - 남성 요인       float64
불임 원인 - 배란 장애       float64
불임 원인 - 자궁내막증       float64
불임 원인 - 정자 농도       float64
배아 생성 주요 이유         float64
총 시술 횟수             float64
클리닉 내 총 시술 횟수       float64
IVF 시술 횟수           float64
DI 시술 횟수            float64
총 임신 횟수             float64
IVF 임신 횟수           float64
DI 임신 횟수            float64
총 출산 횟수             float64
IVF 출산 횟수           float64
DI 출산 횟수            float64
총 생성 배아 수           float64
미세주입된 난자 수          float64
미세주입에서 생성된 배아 수     float64
이식된 배아 수            float64
미세주입 배아 이식 수        float64
저장된 배아 수            float64
미세주입 후 저장된 배아 수     

In [51]:
# float64가 아닌 컬럼만 출력
non_float_columns = X_train.select_dtypes(exclude=['float64']).columns
print("float64가 아닌 컬럼:", list(non_float_columns))

float64가 아닌 컬럼: []


In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)  # k-Nearest Neighbors 기반
X_train[:] = imputer.fit_transform(X_train)
X_val[:] = imputer.transform(X_val)
test[:] = imputer.transform(test)

In [ ]:
def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
        df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
    return df

numeric_columns = X_train.select_dtypes(include=['number']).columns
X_train = remove_outliers(X_train, numeric_columns)
X_val = remove_outliers(X_val, numeric_columns)
test = remove_outliers(test, numeric_columns)

In [ ]:
from scipy import stats

def remove_outliers_zscore(df, columns, threshold=3):
    for col in columns:
        z_scores = np.abs(stats.zscore(df[col]))
        df[col] = np.where(z_scores > threshold, df[col].median(), df[col])
    return df

X_train = remove_outliers_zscore(X_train, numeric_columns)
X_val = remove_outliers_zscore(X_val, numeric_columns)
test = remove_outliers_zscore(test, numeric_columns)

In [ ]:
# 소프트 투표 분류기
voting_clf = VotingClassifier(estimators=models, voting='soft')

# 모델 학습
voting_clf.fit(X_train, y_train)


In [ ]:
y_pred = voting_clf.predict(X_val)


In [ ]:
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.4f}')


### Train

### Predict

In [32]:
pred_proba = voting_clf.predict_proba(test)[:, 1]
sample_submission = pd.read_csv('/content/drive/MyDrive/open/sample_submission.csv')
sample_submission['probability'] = pred_proba
sample_submission.to_csv('./voting_soft_KNN_outlier_zscore__1.csv', index=False)


### Submission

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/open/sample_submission.csv')
sample_submission['probability'] = pred_proba

In [ ]:
sample_submission.to_csv('./baseline_submit_1.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# 데이터 로드 및 ID 컬럼 제거
train = pd.read_csv('/content/drive/MyDrive/open/train.csv').drop(columns=['ID'])
test = pd.read_csv('/content/drive/MyDrive/open/test.csv').drop(columns=['ID'])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
모델 학습 완료!


결측값 70%이상 되는 피쳐 지움
Ordinal Encoder
ExtraTree Classifier